# `II. Field Simulation` 
This .pynb file reads `./inter_results/mesh_result.pkl` generated by `./I_Mesh_Processing.ipynb`, and simulates electric field distribution. The result is stored in `./inter_results/field_result.pkl`.

In this file we compute electric field or potential on some spatial points(called `grid`) for all electrode voltage configurations(i.e. one eletrode is 1V and the rest 0V).Major calculations calls `fastlap` C library which uses a pre-conditioned, adaptive, multipole-accelerated algorithm for solving Laplace problem. Two parameters control multipole acceleration.
+ num_mom, the number of multipole
+ num_lev, the number of levels in the hierarchical spatial decomposition.  
num_lev=1 means direct computation without multipole acceleration. See fastlap ug.pdf and README.rst.


In `run_job` function, `job` is `Configuration` instance and `grid` is discretirized spatial grid (not the mesh). The general workflow (also the routine of BEM method) are:  
1. `solve_singularities()` solves charge distributions by iterative methods to make it consistent with one electrode at 1V and others at 0V (unit potentials). `adapt_mesh()` refines meshes adaptively to achieve certain precision while solving sigulartities.
2. Compute potentials on given grid points by `simulate()`, based on the charge distributions gotten previously.
3. Potential data of each unit potential are saved seperately to a `Result` instance, and also export to VTK files.
4. Return total accumulated charge per electrode in the end.

notes:
* here we invoke `multiprocessing.set_start_method("fork")` for compatibility with python version 3.9
* This is the most time consuming part in our workflow. Running following codes in a `.py` file instead of `.ipynb` may be helpful. Closing other apps in your laptop is helpful. Using HPC is also an option. 
* for reference, this file in my laptop(macbook pro, 2 physical kernels) runs 800s
* This `.ipynb` file will generate a `./.vtk` file for intermediate files, which can be simply ignored.

## (1) import the python packages needed for the script



In [39]:
from time import time
import pickle
import numpy as np
import os
import pyvista as pv
import sys
import vtk
sys.path.append('/home/sqip/Documents/github/bem')
from bem import Electrodes, Sphere, Mesh, Grid, Configuration, Result
import multiprocessing 
# multiprocessing.set_start_method("fork")
from utils.helper_functions import *

## (2) import the mesh saved from I_Mesh_Processing
This should correspond to the same name was exported during mesh_processing. The reason this is even an option is in case you want to load in a different saved mesh. 

In [40]:
radius= 500e-3
area =1e-4
file = 'htrap_el2'
file_in_name = 'inter_results/htrap/'+file+'_'+str(radius)+'_'+str(area)+'.pkl'
file_out_name = 'inter_results/htrap/'+file+'_'+str(radius)+'_'+str(area)+'_simulation'
vtk_out = "inter_results/htrap/.vtks/"+file

# file_out = 'htrap_overhang_taper_el3'
# stl_path = 'inter_results/taper_overhang/'+file+'.stl'
# file_in_name = 'inter_results/taper_overhang/'+file_out+'_'+str(radius)+'_'+str(area)+'.pkl'
# file_out_name = 'inter_results/taper_overhang/'+file_out+'_'+str(radius)+'_'+str(area)+'_simulation_tall_rough'
# open the mesh that will be used for simulation
with open(file_in_name,'rb') as f:
    mesh_unit,xl,yl,zl,mesh,electrode_names= pickle.load(f) # import results from mesh processing

## (2) define grid
Here, we define some spatial points for simulation field, called 'grid'.
The grid is typically a 3-d cube centered on the trapping location.
* s is the unit step for the spatial points
* Lx, Ly, Lz are the dimensions of the 3-d cube
* sx, sy, sz are the step size along each dimension
* nx, ny, nz are the number of points to be simulated along each dimension
* `Grid` is the function imported from the bem package that will be inputted into the simluation

In [41]:
with open(file_in_name,'rb') as f:
    mesh_unit,xl,yl,zl,mesh,electrode_names= pickle.load(f) # import results from mesh processing
    print("zl: ", zl)
# grid to evalute potential and fields atCreate a grid in unit of scaled length mesh_unit. Only choose the interested region (trap center) to save time.
Lx, Ly, Lz = 11*1e-3,11*1e-3,11*1e-3# in the unit of scaled length mesh_unit
# xl,yl,zl = -3.75*1e-3,72*1e-3,270*1.0e-3
xl,yl,zl = 3.75e-3,75e-3,-450e-3
s = 1e-3
sx,sy,sz = s,s,s
print("done")
# ni is number of grid points, si is step size. To  on i direction you need to fix ni*si.
nx, ny, nz = [int(Lx/sx),int(Ly/sy),int(Lz/sz)]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(xl,yl,zl), step=(sx, sy, sz), shape=(nx,ny,nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("lowval",grid.indices_to_coordinates([0,0,0]))
print("Grid center index", grid.indices_to_coordinates((nx/2,ny/2,nz/2)))
print("gridpts:",nx*ny*nz)
center = (xl,yl,zl)
step = (sx,sy,sz)
shape = (nx,ny,nz)
# xyz = np.array([np.linspace(c-s*(h-1)/2., c+s*(h-1)/2.,h)
#                 for c, s, h in zip(center, step, shape)])

zl:  -0.45
done
Size/l: 0.011 0.011 0.011
Step/l: 0.001 0.001 0.001
Shape (grid point numbers): 11 11 11
lowval [-0.00125  0.07    -0.455  ]
Grid center index [ 0.00425  0.0755  -0.4495 ]
gridpts: 1331


## (2) run jobs
`pmap.multiprocessing.Pool(processes).map(job,list)` is used to do several simulations in parallel (depends what you define as parallel). 

`Pool(number)` creates a set of worker processes, called a pool, to submit tasks to. When a job is submitted to a worker process, the worker process executes that task. <b>Each worker process is associated with its own CPU core</b> which means the number of worker processes in the pool is the number of parallel computations that can be performed. <b>number</b> is the number of worker processes to start. 

In other words, if you want to use only 1 core at a time, write Pool(1), if you want 2 cores write Pool(2). If no number is given, `Pool` will create as many worker processes as there are cores on the computer. This speeds things up (you can use all cores simulatneously), but it is important to point out that since each worker process will be requesting RAM, this will consume more memory on your computer than Pool(1). If the total RAM requested exceeds what your computer has, <b>you're gonna have a bad time</b>. One of two things will likely happen- first is that your computer will try to start using 'swap space' which is using disk memory as RAM which will slow down the computation significantly- this is not obvious unless you are looking at the resources being used by your computer (look for normal memory used and swap memory used on system monitor). Second is that your computer wil crash because the worker processes have consumed all the memory. 

`Pool.map`(<b>job</b>,<b>jobs_list</b>) takes two arguments:
* <b>job</b> is the python function that will be turned into a job to be executed by the worker processes
* <b>jobs_list</b> is a list where each item in the list is a set of arguments to be provided to the function when creating a job

The worker processes then execute any job sent to the pool. 

In [42]:
jobs = list(Configuration.select(mesh,'DC.*','RF'))    # select() picks one electrode each time.
# run the different electrodes on the parallel pool
pmap = multiprocessing.Pool(2).map # parallel map
#pmap = map # serial map
t0 = time()
# range(len(jobs))
def run_map():
    pmap(run_job, ((jobs[i], grid, vtk_out,i,len(jobs)) for i in np.arange(len(jobs))))
    print( "Computing time: %f s"%(time()-t0))
    # run_job casts a word after finishing ea"ch electrode.

run_map()

Computing time: 3.842970 s


## (3) save simulation results

In [43]:
electrode_names = ['DC1','DC2','DC3','DC4','DC5','DC6','DC7','DC8','DC9','DC10',
                   'DC11','DC12','DC13','DC14','DC15','DC16','DC17','DC18','DC19','DC20','DC21',
                   'RF']
write_pickle(vtk_out,file_out_name,grid,electrode_names)

FileNotFoundError: File (c:\Users\Photonics\Documents\GitHub\integrated_photonics_bem\examples\Integrated_photonics\inter_results\htrap\.vtks\htrap_el2_DC1_mesh.vtk) not found

In [ ]:
result = Result.load(vtk_out, 'DC1', 'vtk')
print(vtk_out)

FileNotFoundError: File (c:\Users\Photonics\Documents\GitHub\integrated_photonics_bem\examples\Integrated_photonics\inter_results\htrap\.vtks\htrap_el2_DC1_mesh.vtk) not found

In [7]:
length_scale = 1e-3
epsilon_0 = 8.85e-12
electrode_index = np.where(result.configuration.potentials==1)[0][0]
electrode_capacitance_cgs = result.configuration.collect_charges()[electrode_index]
electrode_capacitance_si = 4*np.pi*epsilon_0*length_scale
electrode_capacitance_picofarads = electrode_capacitance_si*1e12
print(electrode_capacitance_picofarads)

0.11121237993707869


## (4) view simulation results
The results can be viewed in an interactive 3-d plot using the package pyvista

In [8]:
"the last simulated mesh was for mesh radius 500e-3mm and max triangle area 3.125e-5 mm"
ele = 'DC16'
data_name = "%s_%s.vtk" % (vtk_out, ele)
data = pv.UniformGrid(data_name)
scalar_name = 'potential'
avals = data[scalar_name]
range = avals.max()-avals.min()
input_range = [avals.min(),avals.min()+range]
Result.view(vtk_out, ele) # add electrode name between '' for result view

c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:873: PyVistaDeprecationWarning: `UniformGrid` is deprecated. Use `ImageData` instead.
  warnings.warn(


In [12]:
# for x in np.arange(0,4):
data_name = "%s_%s.vtk" % (vtk_out, 'RF')
import pyvista as pv 
data = pv.UniformGrid(data_name)
scalar_name = 'potential'

avals = data[scalar_name]
input_range = [-0.2,1.0]
Result.view(vtk_out, 'RF') # add electrode name between '' for result view

c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:873: PyVistaDeprecationWarning: `UniformGrid` is deprecated. Use `ImageData` instead.
  warnings.warn(


In [ ]:


plotter = pv.Plotter(notebook = False)
prefix = vtk_out
name = 'DC15'
base_mesh_name = "%s_mesh.vtk" % prefix
mesh_name = "%s_%s_mesh.vtk" % (prefix, name)

def callback_func(electrode_name):
    idc_name = np.where(mesh.cell_data['electrode_name'] == electrode_name)
    selected_faces = mesh.faces.reshape(-1, 4)[idc_name]
    selected_mesh = pv.PolyData(mesh.points, faces = selected_faces)
    plotter.add_mesh(selected_mesh, name = 'select', style = 'wireframe', line_width=5, color='black')

if os.access(mesh_name, os.R_OK):
    mesh = pv.PolyData(mesh_name)
    charge_min = mesh.cell_data['charge'].min()
    charge_max = mesh.cell_data['charge'].max()
    charge_absmax = abs(mesh.cell_data['charge']).max()

    Ncolors = len(pv.LookupTable('bwr').values)
    Ncolors_half = int(Ncolors/2)
    mid_to_left = int(np.rint(charge_min/charge_absmax * Ncolors_half))
    mid_to_right = int(np.rint(charge_max/charge_absmax * Ncolors_half))
    cmap_values = pv.LookupTable('bwr').values[Ncolors_half + mid_to_left : Ncolors_half + mid_to_right]
    cmap = pv.LookupTable(values = cmap_values, scalar_range = (charge_min, charge_max))
    # cmap = pv.LookupTable(cmap = 'bwr', scalar_range = (-4, 4))
    plotter.add_mesh(mesh, scalars = 'charge', cmap = cmap, show_edges = True,show_scalar_bar=False)
    electrode_list = list(np.unique(mesh.cell_data['electrode_name']))
    # plotter.add_text_slider_widget(callback_func, electrode_list,  pointa=(0.05, 0.9), pointb=(0.3, 0.9), interaction_event = 'always')
elif os.access(base_mesh_name, os.R_OK):
    mesh = pv.PolyData(base_mesh_name)
    # colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', 
    #           '#2ca02c', '#98df8a', '#d62728', '#ff9896', 
    #           '#9467bd', '#c5b0d5', '#8c564b', '#c49c94', 
    #           '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', 
    #           '#bcbd22', '#dbdb8d', '#17becf', '#9edae5',
    #           # above is colormap tab20 from matplotlib
    #           '#0000FF', '#7FFF00', '#4B0082', '#FF00FF', '#800000', '#FFFF00', '#B8860B', '#F0E68C', '#FFFFFF',
    #           # above is blue, chartreuse, indigo, magenta, maroon, yellow, darkgoldenrod, khaki, white
    #           ]
    plotter.add_mesh(mesh, scalars = 'electrode_name', 
                    scalar_bar_args = {'interactive': True, 'label_font_size': 15}, 
                    # cmap = colors, 
                    show_edges = True)
    electrode_list = list(np.unique(mesh.cell_data['electrode_name']))
    plotter.add_text_slider_widget(callback_func, electrode_list,  pointa=(0.05, 0.9), pointb=(0.3, 0.9), interaction_event = 'always')

data_name = "%s_%s.vtk" % (prefix, name)
if os.access(data_name, os.R_OK):
    data = pv.ImageData(data_name)
    if 'field_square' in data.point_data.keys():
        scalar_name = 'field_square'
    else:
        scalar_name = 'potential'
    iso_surfaces = data.contour(isosurfaces = 1000, scalars = scalar_name
                            )
    plotter.add_mesh(iso_surfaces, cmap = 'viridis', opacity = 1,show_scalar_bar=False)

    # def callback_func_iso(value):
    #     iso_slider = data.contour(isosurfaces = 1, scalars = scalar_name, rng = [value, value])
    #     plotter.add_mesh(iso_slider, name = 'iso', cmap = 'Greys', scalars = scalar_name, opacity = 1, show_scalar_bar = False)
    
    # rng_iso = data.get_data_range(scalar_name)
    # plotter.origin = (3.75e-3,72e-3,-270e-3)
    # plotter.add_slider_widget(
    #         callback=callback_func_iso,
    #         rng=rng_iso,
    #         title='potential/field_square',
    #         # color='dimgray',
    #         pointa=(0.4, 0.9),
    #         pointb=(0.9, 0.9), 
    #         interaction_event = 'always',
    #     )
    # plotter.add_mesh_isovalue(data, cmap = 'Greys', scalars = scalar_name,  pointa=(0.4, 0.9), pointb=(0.9, 0.9))

# plotter.show_bounds()
plotter.add_camera_orientation_widget()
plotter.set_background('black', top='white')
plotter.show()

C:\Users\Photonics\AppData\Local\Temp\ipykernel_9304\71528197.py:17: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  charge_absmax = abs(mesh.cell_data['charge']).max()
